## Chapter 6 - Exercise 3: NBA Players

#### Cho dữ liệu nba_2013.csv
#### Sử dụng thuật toán Decision Tree để dự đoán số điểm (points) mà các cầu thủ NBA ghi được trong mùa giải 2013-2014.
#### Mỗi hàng trong dữ liệu chứa thông tin về player thực hiện trong mùa giải 2013-2014 NBA. 
- (với player -- tên player/ pos -- vị trí của player/ g -- số trận mà player đã tham gia/ gs -- số trận mà
player đã bắt đầu/ pts -- tổng số point mà player đã ghi được)
1. Đọc dữ liệu và gán cho biến data. Xem thông tin data: shape, type, head(), tail(), info. Tiền xử
lý dữ liệu (nếu cần)
2. Tạo inputs data với các cột không có giá trị null trừ cột 'player', 'bref_team_id', 'season',
'season_end', 'pts', và outputs data với 1 cột là 'pts' => Vẽ biểu đồ quan sát mối liên hệ giữa
inputs và outputs data
3. Từ inputs data và outputs data => Tạo X_train, X_test, y_train, y_test với tỷ lệ 80:20
4. Thực hiện Decision Tree với X_train, y_train
5. Dự đoán y từ X_test => so sánh với y_test
6. Xem kết quả => Nhận xét model
7. Ghi model nếu model phù hợp

In [32]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeRegressor
%load_ext nb_black
%matplotlib inline


The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [22]:
df = pd.read_csv("data/nba_2013.csv")
df

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,Tony Wroten,SG,20,PHI,72,16,1765,345,808,0.427,...,159,228,217,78,16,204,151,939,2013-2014,2013
477,Nick Young,SG,28,LAL,64,9,1810,387,889,0.435,...,137,166,95,46,12,95,156,1144,2013-2014,2013
478,Thaddeus Young,PF,25,PHI,79,78,2718,582,1283,0.454,...,310,476,182,167,36,165,213,1417,2013-2014,2013
479,Cody Zeller,C,21,CHA,82,3,1416,172,404,0.426,...,235,353,92,40,41,87,170,490,2013-2014,2013


<IPython.core.display.Javascript object>

### Exploratory Data Anlasis (EDA)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   player        481 non-null    object 
 1   pos           481 non-null    object 
 2   age           481 non-null    int64  
 3   bref_team_id  481 non-null    object 
 4   g             481 non-null    int64  
 5   gs            481 non-null    int64  
 6   mp            481 non-null    int64  
 7   fg            481 non-null    int64  
 8   fga           481 non-null    int64  
 9   fg.           479 non-null    float64
 10  x3p           481 non-null    int64  
 11  x3pa          481 non-null    int64  
 12  x3p.          414 non-null    float64
 13  x2p           481 non-null    int64  
 14  x2pa          481 non-null    int64  
 15  x2p.          478 non-null    float64
 16  efg.          479 non-null    float64
 17  ft            481 non-null    int64  
 18  fta           481 non-null    

<IPython.core.display.Javascript object>

In [24]:
"""
=> có null, sử dụng interpolate để giải quyết
"""

'\n=> có null, sử dụng interpolate để giải quyết\n'

<IPython.core.display.Javascript object>

In [25]:
df_new = df.interpolate()

<IPython.core.display.Javascript object>

In [26]:
# profile = ProfileReport(df_new)
# profile

<IPython.core.display.Javascript object>

In [20]:
#profile.to_file("Exercise3_Exploratory_Data_Analysis.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [29]:
X = df_new.drop(columns=["pts"])
y = df_new.pts

<IPython.core.display.Javascript object>

In [34]:
X_new = pd.get_dummies(data=X)

<IPython.core.display.Javascript object>

### Applying machine learning algorithm

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    X_new, y, test_size=0.2, random_state=42
)

<IPython.core.display.Javascript object>

In [36]:
# vì plt (tarhet) là continuous variable => sử dụng decision tree regressor
model = DecisionTreeRegressor().fit(X_train, y_train)

<IPython.core.display.Javascript object>

### Đánh giá

In [37]:
# Kiểm tra độ chính xác
from sklearn import metrics

print("The Train/ Score is: ", model.score(X_train, y_train) * 100, "%")
print("The Test/ Score accuracy is: ", model.score(X_test, y_test) * 100, "%")
y_pred = model.predict(X_test)
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))

The Train/ Score is:  100.0 %
The Test/ Score accuracy is:  98.58860124960313 %
Mean Squared Error: 3010.814432989691


<IPython.core.display.Javascript object>

In [38]:
"""
Training và Testing cùng có R^2 cao và gần bằng nhau
Mô hình trên cho R^2 cao ~ 0.99, cho thấy nó fit 99% dữ liệu
MSE vừa phải => mô hình phù hợp
"""

'\nTraining và Testing cùng có R^2 cao và gần bằng nhau\nMô hình trên cho R^2 cao ~ 0.99, cho thấy nó fit 99% dữ liệu\nMSE vừa phải => mô hình phù hợp\n'

<IPython.core.display.Javascript object>